# Step 04 - Deploy model to Online Endpoint

Use custom serving container to deploy model to managed endpoint with model data collection (inputs & outputs) configured for automatic capture of data sent to the endpoint

### Import required packages and connect to AML workspace

In [ ]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, Model, Environment, ModelPackage, BaseEnvironment, AzureMLOnlineInferencingServer, CodeConfiguration, DataCollector, DeploymentCollection
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import ContainerRegistryCredential


subscription_id = "..."
resource_group = "..."
workspace = "..."
model_name = "zone1-power-consumption-xgboost-model"

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)
ml_client

### Retrieve container registry name 

Note: this can be overwritten if targeting alternative container registries

In [ ]:
ml_client.workspaces.get(ml_client.workspace_name).container_registry

acr_name = ml_client.workspaces.get(ml_client.workspace_name).container_registry.split('/')[-1]
acr_name

acr_login_server = f'{acr_name}.azurecr.io'  # ACR login server

image_name = 'zone-power-consumption:2' 

container_path = f'{acr_login_server}/{image_name}'
container_path

In [3]:
env = Environment(
    image=container_path,
)

In [4]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "power-endpoint"

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="DEMO - Endpoint for Power Consumption Model Demo",
    auth_mode="key",
)
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

### Create deployment which uses custom scoring script (`score.py`) for input/output logging

In [ ]:
model = Model(name="power-model", path="model")

inputs_collection = DeploymentCollection(enabled=True)
outputs_collection = DeploymentCollection(enabled=True)

mdc = DataCollector(collections={'model_inputs':inputs_collection, 'model_outputs':outputs_collection})

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    environment_variables={
        "MODEL_BASE_PATH": "/",
        "MODEL_NAME": "power-model",
    },
    instance_type="Standard_F2s_v2",
    instance_count=1,
    data_collector=mdc,
    code_configuration=CodeConfiguration(code=".", scoring_script="score.py")
)
ml_client.begin_create_or_update(blue_deployment).result()

In [16]:
endpoint.traffic = {"blue": 100}
endpoint.mirror_traffic = {}
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
